# https://keras.io/keras_tuner/

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.duplicated().sum()

0

In [4]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [42]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense, Dropout

In [11]:
model = Sequential()
model.add(Dense(32, input_dim = 8, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [12]:
model.fit(X_train,y_train, epochs = 100, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.8252 - accuracy: 0.3697 - val_loss: 0.7488 - val_accuracy: 0.4156
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7350 - accuracy: 0.4707 - val_loss: 0.6831 - val_accuracy: 0.6299
Epoch 3/100
20/20 [==============================] - 0s 13ms/step - loss: 0.6688 - accuracy: 0.6254 - val_loss: 0.6368 - val_accuracy: 0.6688
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6229 - accuracy: 0.6547 - val_loss: 0.6020 - val_accuracy: 0.6883
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5867 - accuracy: 0.6922 - val_loss: 0.5747 - val_accuracy: 0.7338
Epoch 6/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5607 - accuracy: 0.7117 - val_loss: 0.5568 - val_accuracy: 0.7338
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.5402 - accuracy: 0.7296 - val_loss: 0.5391 - val_accuracy: 0.7727
Epoch 8/10

In [13]:
!pip install keras-tuner

In [14]:
import keras_tuner as kt

# Optimizing Optimizer

In [15]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, input_dim = 8, activation= "relu"))
  model.add(Dense(1, activation = "sigmoid"))

  optimizer = hp.Choice("optimizers", values = ["adam","adadelta","rmsprop", "sgd"])
  model.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = optimizer)

  return model

In [16]:
tuner = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 5
)

Reloading Tuner from ./untitled_project/tuner0.json


In [17]:
tuner.search(X_train,y_train, epochs = 5, validation_data = (X_test, y_test))

In [18]:
tuner.get_best_hyperparameters()

In [19]:
tuner.get_best_hyperparameters()[0]

In [20]:
tuner.get_best_hyperparameters()[0].values

{'optimizers': 'rmsprop'}

In [21]:
model = tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(X_train,y_train, batch_size= 32, epochs = 100, initial_epoch=6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 26ms/step - loss: 0.5140 - accuracy: 0.7492 - val_loss: 0.5316 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5036 - accuracy: 0.7573 - val_loss: 0.5250 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4952 - accuracy: 0.7638 - val_loss: 0.5191 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4885 - accuracy: 0.7655 - val_loss: 0.5145 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4830 - accuracy: 0.7638 - val_loss: 0.5110 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4787 - accuracy: 0.7736 - val_loss: 0.5077 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4748 - accuracy: 0.7638 - val_loss: 0.5047 - val_accuracy: 0.7922
Epoch 14

# Optimizing no. of units

In [24]:
def build_model(hp):
  model = Sequential()
  units = hp.Int("units",min_value = 8, max_value = 128,step = 32)
  model.add(Dense(units = units , activation = "relu"))
  model.add(Dense(1,activation = "sigmoid"))

  model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return model

In [25]:
tuner  = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 5,
    directory = "mydirectory",
    project_name = "units"
)

Reloading Tuner from mydirectory/units/tuner0.json


In [26]:
tuner.search(X_train,y_train,epochs = 5, validation_data = (X_test, y_test))

In [27]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [28]:
model = tuner.get_best_models(num_models=1)[0]

In [30]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6, validation_data = (X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 15ms/step - loss: 0.4831 - accuracy: 0.7752 - val_loss: 0.4829 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4742 - accuracy: 0.7850 - val_loss: 0.4792 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4660 - accuracy: 0.7818 - val_loss: 0.4741 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4600 - accuracy: 0.7818 - val_loss: 0.4713 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4556 - accuracy: 0.7818 - val_loss: 0.4706 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4525 - accuracy: 0.7883 - val_loss: 0.4677 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4497 - accuracy: 0.7883 - val_loss: 0.4682 - val_accuracy: 0.7857
Epoch 14

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 1)                 105       
                                                                 
Total params: 1041 (4.07 KB)
Trainable params: 1041 (4.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Optimizating No of hidden layers

In [35]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(104, input_dim = 8, activation = "relu"))

  for i in range(hp.Int("num_layers",min_value = 1, max_value = 10)):
    model.add(Dense(104,activation = "relu"))
  model.add(Dense(1,activation = "sigmoid"))

  model.compile(optimizer="adam",loss = "binary_crossentropy", metrics = ["accuracy"])

  return model

In [36]:
tuner = kt.RandomSearch(
    build_model,
    max_trials = 5,
    objective = "accuracy",
    directory = "mydirectory",
    project_name = "num_layers"
)

In [37]:
tuner.search(X_train,y_train, epochs = 5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 05s]
accuracy: 0.7882736325263977

Best accuracy So Far: 0.7996742725372314
Total elapsed time: 00h 00m 21s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

In [40]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, initial_epoch = 5, validation_data =(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 3s 22ms/step - loss: 0.4103 - accuracy: 0.8127 - val_loss: 0.4988 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4108 - accuracy: 0.8241 - val_loss: 0.4923 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3970 - accuracy: 0.8306 - val_loss: 0.5118 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3837 - accuracy: 0.8355 - val_loss: 0.4956 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3608 - accuracy: 0.8339 - val_loss: 0.4943 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3754 - accuracy: 0.8290 - val_loss: 0.4899 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3363 - accuracy: 0.8648 - val_loss: 0.5267 - val_accuracy: 0.7792
Epoch 13/

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 104)               10920     
                                                                 
 dense_2 (Dense)             (None, 104)               10920     
                                                                 
 dense_3 (Dense)             (None, 104)               10920     
                                                                 
 dense_4 (Dense)             (None, 104)               10920     
                                                                 
 dense_5 (Dense)             (None, 104)               10920     
                                                                 
 dense_6 (Dense)             (None, 104)               1

# optimizing no of units, activation, learning_rate

In [43]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Tune number of units.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            # Tune the activation function to use.
            activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )
    # Tune whether to use dropout.
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1, activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [44]:
tuner = kt.RandomSearch(
    build_model,
    max_trials = 5,
    objective = "accuracy",
    directory = "mydirectory",
    project_name = "nodes_activation_lr"
)

In [45]:
tuner.search(X_train,y_train, epochs = 5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 03s]
accuracy: 0.34690552949905396

Best accuracy So Far: 0.34690552949905396
Total elapsed time: 00h 00m 13s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'units': 416,
 'activation': 'relu',
 'dropout': False,
 'lr': 0.0010514440542815132}

In [47]:
model = tuner.get_best_models(num_models=1)[0]

In [48]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, initial_epoch = 5, validation_data =(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.3469 - val_loss: 0.0000e+00 - val_accuracy: 0.3571
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.3469

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 416)               3744      
                                                                 
 dense_1 (Dense)             (None, 1)                 417       
                                                                 
Total params: 4161 (16.25 KB)
Trainable params: 4161 (16.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# optimizing all the hyperparametrs

In [66]:
def build_model(hp):

  model = Sequential()

  counter = 0
  for i in range(hp.Int("num_layers", min_value = 1, max_value = 8)):

    if counter == 0:
      model.add(Dense(hp.Int("nodes" + str(i), min_value = 8, max_value = 512,step =8), activation = hp.Choice("activation" + str(i), values = ["relu","tanh"])))
      model.add(Dropout(hp.Choice("droppout" + str(i), [0.1,0.2,0.3,0.4,0.5,0.6])))
    else:
      model.add(Dense(hp.Int("nodes" + str(i), min_value = 8, max_value = 512, step = 8), activation = hp.Choice("activation" + str(i), values = ["relu","tanh"])))
      model.add(Dropout(hp.Choice("droppout" + str(i), [0.1,0.2,0.3,0.4,0.5,0.6])))

    counter += 1

  model.add(Dense(1,activation='sigmoid'))
  learning_rate = hp.Float("lr", min_value=1e-3, max_value=1, sampling="log")
  optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','nadam','adadelta'])

  model.compile(optimizer = optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'],
                  )
  model.learning_rate = learning_rate

  return model


In [67]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory="mydir",
                        project_name='final')

In [68]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 59s


In [69]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'nodes0': 344,
 'activation0': 'relu',
 'droppout0': 0.6,
 'lr': 0.1362601717791948,
 'optimizer': 'rmsprop',
 'nodes1': 8,
 'activation1': 'relu',
 'droppout1': 0.1,
 'nodes2': 8,
 'activation2': 'relu',
 'droppout2': 0.1,
 'nodes3': 8,
 'activation3': 'relu',
 'droppout3': 0.1,
 'nodes4': 8,
 'activation4': 'relu',
 'droppout4': 0.1,
 'nodes5': 8,
 'activation5': 'relu',
 'droppout5': 0.1}

In [70]:
model = tuner.get_best_models(num_models=1)[0]

In [71]:
model.fit(X_train,y_train,epochs=300,initial_epoch=10,validation_data=(X_test,y_test))

Epoch 11/300
20/20 [==============================] - 3s 17ms/step - loss: 0.5171 - accuracy: 0.7329 - val_loss: 0.4816 - val_accuracy: 0.8117
Epoch 12/300
20/20 [==============================] - 0s 6ms/step - loss: 0.5039 - accuracy: 0.7410 - val_loss: 0.4793 - val_accuracy: 0.8182
Epoch 13/300
20/20 [==============================] - 0s 6ms/step - loss: 0.5020 - accuracy: 0.7590 - val_loss: 0.4737 - val_accuracy: 0.8182
Epoch 14/300
20/20 [==============================] - 0s 6ms/step - loss: 0.4980 - accuracy: 0.7313 - val_loss: 0.4705 - val_accuracy: 0.8247
Epoch 15/300
20/20 [==============================] - 0s 6ms/step - loss: 0.4869 - accuracy: 0.7280 - val_loss: 0.4758 - val_accuracy: 0.8182
Epoch 16/300
20/20 [==============================] - 0s 6ms/step - loss: 0.4903 - accuracy: 0.7508 - val_loss: 0.4734 - val_accuracy: 0.8182
Epoch 17/300
20/20 [==============================] - 0s 7ms/step - loss: 0.5113 - accuracy: 0.7427 - val_loss: 0.4786 - val_accuracy: 0.8182
Epoch

In [72]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 344)               3096      
                                                                 
 dropout (Dropout)           (None, 344)               0         
                                                                 
 dense_1 (Dense)             (None, 8)                 2760      
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 8)                 7